In [14]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from typing import List
from operator import itemgetter

# LangChain Imports needed
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.runnables import ConfigurableField


# Our own libraries needed
# from common.prompts import DOCSEARCH_PROMPT
# from common.utils import get_search_results

from dotenv import load_dotenv
load_dotenv(".env")

Python-dotenv could not parse statement starting at line 35


True

In [15]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['searchkey']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

In [16]:
# Text-based Indexes that we are going to query (from Notebook 01 and 02)
index1_name = os.environ['AZURE_SEARCH_INDEX_NAME']
indexes = [ index1_name]

In [17]:
QUESTION = "Tell me about how to dertermin distribution of discount store network."

In [22]:
agg_search_results = dict()
k = 10

for index in indexes:
    search_payload = {
        "search": QUESTION, # Text query
        "select":  "chunk_id, title,  chunk, text_vector, image_vector",   #"id,
        "queryType": "semantic",
        "vectorQueries": [{"text": QUESTION, "fields": "text_vector", "kind": "text", "k": k, 
                           "threshold": { 
                                 "kind": "vectorSimilarity", 
                                 "value": 0.8 
                              }
                          }], # Vector query
        "semanticConfiguration": "vector-1725849931807-semantic-configuration",  ### this comes from JSON viewable at INDEXES's "Edited JSON"
        "captions": "extractive",
        "answers": "extractive",
        "count":"true",
        "top": k
    }

    r = requests.post(os.environ['searchservice'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    # print(' --- ',search_results)
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

200
Index: vector-1725849931807 Results Found: 195, Results Returned: 10


In [ ]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():

    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))

            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['chunk_id']]={
                                    "title": result['title'],
                                    "chunk": result['chunk'],         
                                    
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in content :   #sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['title']
    chunk = str(ordered_content[id]['chunk']) if (ordered_content[id]['chunk']) else ordered_content[id]['chunk']
    # score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5>' + title + ' : '+chunk+ '</h5>'))